In [1]:
# =============================================================================
# CELL 1: IMPORTS WITH WINDOWS/VPN FIXES
# =============================================================================

import sys
import warnings

# Suppress noisy warnings
warnings.filterwarnings("ignore")

# === WINDOWS ASYNC FIX ===
if sys.platform == "win32":
    import asyncio

    try:
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        print("✅ Windows async policy set")
    except Exception as e:
        print(f"⚠️ Async policy warning: {e}")

# === SSL/VPN FIXES ===
import os

os.environ["PYTHONHTTPSVERIFY"] = "0"
os.environ["CURL_CA_BUNDLE"] = ""

import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === MAIN IMPORTS ===
import pandas as pd
import numpy as np
import requests
import io
import json
import time

# === YAHOO IMPORTS (with error handling) ===
try:
    from yahooquery import Ticker

    print("✅ yahooquery loaded")
except Exception as e:
    print(f"❌ yahooquery failed: {e}")
    Ticker = None

try:
    import yfinance as yf

    print("✅ yfinance loaded")
except Exception as e:
    print(f"❌ yfinance failed: {e}")
    yf = None

print("\n🎉 All imports complete!")

✅ Windows async policy set
✅ yahooquery loaded
✅ yfinance loaded

🎉 All imports complete!


In [2]:
# =============================================================================
# CELL 2: CONFIGURATION AND SETUP
# =============================================================================
# This cell sets up all the "rules" our stock screener will follow.


import os
import json

# ==========================================
# CONFIGURATION - These are our "settings"
# ==========================================

# 1. FOLDER SETUP
# ---------------
# We need a place to save our results. This creates a folder called "YfinanceDataDump"

DATA_FOLDER = "YfinanceDataDump"

# Check if the folder already exists. If not, create it.
# os.path.exists() returns True if the folder exists, False if it doesn't
if not os.path.exists(DATA_FOLDER):
    try:
        # os.makedirs() creates the folder
        os.makedirs(DATA_FOLDER)
        print(f"Created data folder: {DATA_FOLDER}")
    except Exception as e:
        # If something goes wrong (like no permission), save to current folder
        print(
            f"Warning: Could not create folder '{DATA_FOLDER}'. Saving to current directory. Error: {e}"
        )
        DATA_FOLDER = "."  # The dot means "current folder"

# 2. FILE PATHS
# -------------
# These are the names of the files where we'll save our results
# os.path.join() combines the folder name with the file name properly
# (handles / vs \ on different operating systems automatically)
CACHE_FILE = os.path.join(
    DATA_FOLDER, "financial_cache_ca.json"
)  # Stores data we've already fetched
FORTRESS_CSV = os.path.join(
    DATA_FOLDER, "fortress_stocks_ca.csv"
)  # Best quality stocks
STRONG_CSV = os.path.join(DATA_FOLDER, "strong_stocks_ca.csv")  # Good quality stocks
RISKY_CSV = os.path.join(DATA_FOLDER, "risky_stocks_ca.csv")  # Lower quality stocks
ANALYST_CSV = os.path.join(
    DATA_FOLDER, "Analyst_Fortress_Picks_ca.csv"
)  # Analyst favorites
BUFFETT_CSV = os.path.join(DATA_FOLDER, "Buffett_Value_Picks_ca.csv")  # Value picks

# 3. UNIVERSE FILTERS - Minimum requirements for a stock to be considered
# -----------------------------------------------------------------------


MIN_PRICE = 2.00  # Stock price must be at least $2 (avoid "penny stocks")
MIN_VOLUME = (
    100_000  # At least 100,000 shares traded per day (so we can buy/sell easily)
)
# The underscores are just for readability (100_000 = 100000)
MIN_CAP = 50_000_000  # Market cap at least $50 million (company value)
# Market Cap = Stock Price × Number of Shares
MIN_CURRENT_RATIO = 1.2  # Current Ratio must be > 1.2
# Current Ratio = Current Assets / Current Liabilities
# If > 1, the company can pay its short-term bills
MAX_PE_RATIO = 100.0  # P/E ratio must be less than 100
# P/E = Price / Earnings Per Share


# 4. SAFETY THRESHOLDS - Additional quality filters
# -------------------------------------------------
MIN_INTEREST_COVERAGE = (
    1.5  # Interest Coverage > 1.5 means company earns 1.5x what it owes in interest
)
# Interest Coverage = EBIT / Interest Expense
# If < 1, company can't pay its interest bills = bad!
MIN_ROIC = 0.05  # ROIC (Return on Invested Capital) > 5%
# ROIC = Operating Profit / Capital Invested
# Shows how well management uses money to generate returns
FORTRESS_MARGIN_THRESHOLD = 0.05  # Operating Margin > 5% for "Fortress" (best) tier
# Operating Margin = Operating Income / Revenue
# Shows what % of sales becomes actual profit

# Sectors we want to EXCLUDE (skip)
# Financial Services and Real Estate have very different financial structures
# that don't work well with our filters
EXCLUDED_SECTORS = ["Financial Services", "Real Estate"]

CACHE_EXPIRY_DAYS = 30  # Re-fetch data if our saved data is older than 30 days


# ==========================================
# HELPER FUNCTIONS - Reusable code blocks
# ==========================================
# Functions are like recipes - we define them once, then use them whenever needed


def load_cache():
    """
    Load previously saved financial data from our cache file.

    Why cache? Getting data from Yahoo Finance is slow. If we already
    fetched data recently, we can just load it from our saved file instead.

    Returns:
        dict: A dictionary of saved data, or empty dict {} if no cache exists
    """
    # Check if the cache file exists
    if os.path.exists(CACHE_FILE):
        try:
            # 'r' means "read mode" - we're reading the file, not writing to it
            with open(CACHE_FILE, "r") as f:
                # json.load() reads the JSON file and converts it to a Python dictionary
                return json.load(f)
        except:
            # If something goes wrong reading the file, return empty dictionary
            return {}
    # If file doesn't exist, return empty dictionary
    return {}


def save_cache(cache_data):
    """
    Save our financial data to a file so we can reuse it later.

    Args:
        cache_data (dict): The data we want to save
    """
    try:
        # 'w' means "write mode" - creates the file or overwrites existing
        with open(CACHE_FILE, "w") as f:
            # json.dump() converts the Python dictionary to JSON format and saves it
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")


def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculate the Altman Z-Score - a formula that predicts bankruptcy risk.

    Created by Professor Edward Altman in 1968.
    - Z > 2.99: "Safe Zone" - company is financially healthy
    - Z between 1.81 and 2.99: "Grey Zone" - uncertain
    - Z < 1.81: "Distress Zone" - high risk of bankruptcy

    The Formula: Z = 1.2A + 1.4B + 3.3C + 0.6D + 1.0E

    Where:
    A = Working Capital / Total Assets (liquidity)
    B = Retained Earnings / Total Assets (profitability over time)
    C = EBIT / Total Assets (operating efficiency)
    D = Market Value of Equity / Total Liabilities (market confidence)
    E = Sales / Total Assets (asset efficiency)

    Args:
        bs: Balance Sheet data (DataFrame from yfinance)
        fin: Financial data (DataFrame from yfinance)
        market_cap: Market capitalization in dollars

    Returns:
        float: The Z-Score, or 0 if calculation fails
    """
    try:
        # Helper function to safely get values from DataFrames
        # Sometimes the data has different names, so we try multiple
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    # .iloc[0] gets the first value (most recent year)
                    return df.loc[k].iloc[0]
            return 0

        # Get values from Balance Sheet (bs) and Financials (fin)
        total_assets = get_val(bs, ["Total Assets"])
        total_liab = get_val(
            bs, ["Total Liabilities Net Minority Interest", "Total Liabilities"]
        )
        current_assets = get_val(bs, ["Current Assets", "Total Current Assets"])
        current_liab = get_val(bs, ["Current Liabilities", "Total Current Liabilities"])
        retained_earnings = get_val(bs, ["Retained Earnings"])

        ebit = get_val(
            fin, ["EBIT", "Operating Income"]
        )  # Earnings Before Interest & Taxes
        total_revenue = get_val(fin, ["Total Revenue"])

        # Can't divide by zero, so return 0 if missing key data
        if total_assets == 0 or total_liab == 0:
            return 0

        # Calculate each component of the Z-Score formula
        # A: Working Capital (can company pay short-term bills?)
        A = (current_assets - current_liab) / total_assets

        # B: Retained Earnings (accumulated profits over the years)
        B = retained_earnings / total_assets

        # C: EBIT (operating profitability)
        C = ebit / total_assets

        # D: Market Cap vs Debt (how much does market trust vs owe?)
        D = market_cap / total_liab

        # E: Revenue (is the company using its assets efficiently?)
        E = total_revenue / total_assets

        # The final formula with Altman's coefficients
        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)

    except Exception as e:
        return 0  # Return 0 if any calculation fails

In [3]:
# =============================================================================
# CELL 3: STEP 1 - FETCH CANADIAN STOCK UNIVERSE
# =============================================================================
# This function gets a list of ALL Canadian stocks we want to analyze.
# It tries 3 different methods (in case one fails):
#   1. Official TMX (Toronto Stock Exchange) list
#   2. Wikipedia S&P/TSX list
#   3. Hardcoded backup list


def get_combined_universe():
    """
    Get a list of all Canadian stock tickers to analyze.

    Returns:
        list: A list of ticker symbols like ['RY.TO', 'TD.TO', 'SHOP.TO', ...]
              .TO = Toronto Stock Exchange
              .V = TSX Venture Exchange (smaller companies)
    """
    print("--- STEP 1: Fetching Canadian Universe (TSX & TSX-V) ---")
    tickers = []  # This list will store all our ticker symbols

    # --- METHOD 1: TMX OFFICIAL MOC LIST ---
    # MOC = Market on Close - a list of stocks eligible for special trading
    url_tmx = "https://www.tsx.com/files/trading/moc-eligible-stocks.txt"

    # Headers make our request look like it's from a web browser
    # Some websites block requests that don't have these headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }

    try:
        print("   -> Attempting to fetch official TMX list...")

        # Send a request to the TMX website
        # timeout=10 means give up after 10 seconds if no response
        response = requests.get(url_tmx, headers=headers, timeout=10)

        # raise_for_status() throws an error if the request failed (like 404 error)
        response.raise_for_status()

        # The file is text, so we decode it and split it into lines
        lines = response.content.decode("utf-8").split("\n")

        # Process each line to extract stock symbols
        for line in lines:
            parts = line.strip().split()  # Split by whitespace
            if len(parts) < 3:
                continue  # Skip lines that don't have enough parts

            # The file format varies, so we check where the exchange name is
            symbol = None
            exchange = None

            # Format A: "TSX    RY    ROYAL BANK..."
            if parts[0] in ["TSX", "TSXV"]:
                exchange = parts[0]
                symbol = parts[1]
            # Format B: "RY     ROYAL BANK...     TSX"
            elif parts[-1] in ["TSX", "TSXV"]:
                exchange = parts[-1]
                symbol = parts[0]

            if symbol and exchange:
                # Yahoo Finance uses hyphens, TMX uses dots
                # So BAM.A becomes BAM-A
                clean_symbol = symbol.replace(".", "-")

                # Add the exchange suffix that Yahoo Finance expects
                if exchange == "TSX":
                    tickers.append(f"{clean_symbol}.TO")  # .TO = Toronto
                elif exchange == "TSXV":
                    tickers.append(f"{clean_symbol}.V")  # .V = Venture

        # Remove duplicates using set(), then convert back to list
        tickers = list(set(tickers))
        print(f"   -> Success: Found {len(tickers)} stocks via TMX.")

    except Exception as e:
        print(f"   -> TMX Fetch Failed ({e}). Trying Backup...")

    # --- METHOD 2: WIKIPEDIA S&P/TSX COMPOSITE ---
    # Only try this if Method 1 returned 0 stocks
    if len(tickers) == 0:
        try:
            print("   -> Attempting to scrape S&P/TSX Composite from Wikipedia...")
            url_wiki = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"

            # pd.read_html() finds all tables on a webpage
            # and converts them to DataFrames automatically
            dfs = pd.read_html(url_wiki)

            # Look through all tables to find one with 'Symbol' column
            for df in dfs:
                if "Symbol" in df.columns:
                    wiki_tickers = df["Symbol"].astype(str).tolist()
                    for t in wiki_tickers:
                        t = t.replace(".", "-")  # Fix BAM.A to BAM-A
                        if not t.endswith(".TO"):
                            t = t + ".TO"  # Add Toronto suffix
                        tickers.append(t)
                    break  # Found the right table, stop looking

            tickers = list(set(tickers))  # Remove duplicates
            print(f"   -> Success: Found {len(tickers)} stocks via Wikipedia.")

        except Exception as e:
            print(f"   -> Wikipedia Scraping Failed ({e}).")

    # --- METHOD 3: EMERGENCY FALLBACK LIST ---
    # If both methods failed, use a hardcoded list of major Canadian stocks
    if len(tickers) == 0:
        print("   -> All web fetches failed. Using Emergency Hardcoded List.")
        tickers = [
            "SHOP.TO",
            "RY.TO",
            "TD.TO",
            "CNR.TO",
            "CP.TO",
            "CSU.TO",
            "ATD.TO",
            "BMO.TO",
            "BNS.TO",
            "TRP.TO",
            "ENB.TO",
            "CNQ.TO",
            "BCE.TO",
            "CM.TO",
            "QSR.TO",
            "GIB-A.TO",
            "SU.TO",
            "WCN.TO",
            "TECK-B.TO",
            "T.TO",
            # ... and more major stocks
        ]
        print(f"   -> Loaded {len(tickers)} major stocks.")

    return tickers

In [4]:
# =============================================================================
# CELL 4: STEP 2 - LIGHTWEIGHT FILTER (QUICK SCREENING)
# =============================================================================
# This is the first filter - it quickly eliminates stocks that don't meet
# basic requirements. It's "lightweight" because it uses fast bulk data fetching.


def get_initial_survivors(tickers):
    """
    Filter stocks using basic criteria.
    Args:
        tickers (list): List of all stock symbols to check

    Returns:
        DataFrame: Table of stocks that passed all filters with their metrics
    """
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")

    chunk_size = 500  # Process stocks in batches of 500 at a time
    survivors = []  # This list will hold stocks that pass all filters

    # Split our list into chunks (batches) divides a large request into smaller parts
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    # Process each chunk
    for i, chunk in enumerate(chunks):
        # Print progress every 5 batches
        if i % 5 == 0:
            print(f" -> Processing Batch {i+1}/{len(chunks)}...")

        try:
            # Create a Ticker object for all stocks in this chunk
            # asynchronous=True means it fetches data for multiple stocks simultaneously
            yq = Ticker(chunk, asynchronous=True)

            # Get multiple types of data at once (this is the "bulk fetch")
            # These are different Yahoo Finance data modules
            df_modules = yq.get_modules(
                "summaryProfile summaryDetail financialData price defaultKeyStatistics"
            )

            # Loop through each stock's data
            for symbol, data in df_modules.items():
                # If data is just a string, it means there was an error
                if isinstance(data, str):
                    continue

                try:
                    # Extract the stock price
                    # The .get() method returns the value if it exists, or 0 if it doesn't
                    price = data.get("price", {}).get("regularMarketPrice", 0)
                    if price is None:
                        price = 0

                    # Extract average daily trading volume
                    vol = data.get("summaryDetail", {}).get("averageVolume", 0)
                    if vol is None or vol == 0:
                        # Try alternative location for volume data
                        vol = data.get("price", {}).get("averageDailyVolume10Day", 0)

                    # Extract market capitalization (total company value)
                    cap = data.get("price", {}).get("marketCap", 0)
                    if cap is None:
                        cap = 0

                    # Extract sector (Technology, Healthcare, etc.)
                    sector = data.get("summaryProfile", {}).get("sector", "Unknown")

                    # Get financial data
                    fin_data = data.get("financialData", {})
                    curr_ratio = fin_data.get("currentRatio", 0)  # Current Ratio
                    op_margins = fin_data.get(
                        "operatingMargins", 0
                    )  # Operating Margin (as decimal)
                    if curr_ratio is None:
                        curr_ratio = 0
                    if op_margins is None:
                        op_margins = 0

                    # Get P/E ratio
                    pe = data.get("summaryDetail", {}).get("trailingPE")

                    # ====== APPLY FILTERS ======
                    # Each 'continue' statement skips to the next stock (odd yes but continue skipping to the next stock essentially means it failed the filter and we move on)

                    # Skip if P/E is too high (overvalued)
                    if pe is not None and pe > MAX_PE_RATIO:
                        continue

                    # Skip if price is too low (penny stock)
                    if price < MIN_PRICE:
                        continue

                    # Skip if company is too small
                    if cap < MIN_CAP:
                        continue

                    # Skip if not enough trading volume (hard to buy/sell)
                    if vol < MIN_VOLUME:
                        continue

                    # Skip if in excluded sectors
                    # any() returns True if ANY item in the list is True
                    if any(x in sector for x in EXCLUDED_SECTORS):
                        continue

                    # Skip if current ratio is too low (can't pay bills)
                    if curr_ratio < MIN_CURRENT_RATIO:
                        continue

                    # Skip if operating margin is zero or negative (not profitable)
                    if op_margins <= 0:
                        continue

                    # If we get here, the stock passed ALL filters!
                    # Add it to our survivors list
                    survivors.append(
                        {
                            "Ticker": symbol,
                            "Sector": sector,
                            "Price": price,
                            "Op Margin %": round(
                                op_margins * 100, 2
                            ),  # Convert to percentage
                            "P/E": round(pe, 2) if pe else 0,
                            "Curr Ratio": curr_ratio,
                            "Mkt Cap (B)": round(
                                cap / 1_000_000_000, 2
                            ),  # Convert to billions
                        }
                    )

                except:
                    continue  # Skip this stock if any error occurs

        except:
            continue  # Skip this batch if any error occurs

    # Convert our list of dictionaries to a pandas DataFrame (table)
    return pd.DataFrame(survivors)

In [5]:
# =============================================================================
# CELL 5: STEP 3 - DEEP FINANCIAL ANALYSIS
# =============================================================================
# This is the detailed analysis phase. For each stock that passed Step 2,
# we download detailed financial statements and calculate advanced metrics.


def get_advanced_metrics(survivor_df):
    """
    Perform deep financial analysis on stocks that passed initial screening.


    Args:
        survivor_df: DataFrame of stocks from Step 2

    Returns:
        DataFrame: Stocks with full analysis and tier classification
    """
    tickers = survivor_df["Ticker"].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")

    # Load our cached data (data we've already fetched before)
    cache = load_cache()
    current_time = time.time()  # Current time in seconds since 1970 (Unix timestamp)
    expiry_seconds = (
        CACHE_EXPIRY_DAYS * 86400
    )  # Convert days to seconds (86400 sec/day)

    final_data = []  # Will store our results

    # Loop through each stock
    for i, ticker in enumerate(tickers):
        # Print progress every 20 stocks
        if i % 20 == 0:
            print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")

        # Uncomment the line below if you're getting throttled (too many requests)
        # time.sleep(0.75)  # Wait 0.75 seconds between requests

        def determine_tier_history(metrics, is_fortress_margin, is_pos_margin):
            """
            Determine tier based on Margins AND Financial Health (Z-Score).

            STRICT RULES:
            1. Fortress = High Margin (>5%) AND Safe Z-Score (>2.99)
            2. Strong   = Positive Margin (>0%) AND Acceptable Z-Score (>1.81)
            3. Risky    = Fails either margins or safety
            """

            # Extract Z-Score from the metrics dictionary
            z_val = metrics.get("z_score", 0)

            # 1. IMMEDIATE FAILURES (Hard Safety Stops)
            # If a company can't pay interest or has low return on capital, it's Risky.
            if metrics["int_cov"] < MIN_INTEREST_COVERAGE:
                return "Risky"
            if metrics["roic"] < MIN_ROIC:
                return "Risky"

            # 2. FORTRESS CRITERIA (The "Perfect" Stock)
            # Must have BOTH strong margins AND a safe Z-Score
            if is_fortress_margin and z_val >= 2.99:
                return "Fortress"

            # 3. STRONG CRITERIA (The "Good" Stock)
            # Must have at least positive margins AND be out of the "Distress Zone"
            # We use 1.81 because Z < 1.81 implies high bankruptcy risk
            elif is_pos_margin and z_val >= 1.81:
                return "Strong"

            # 4. FALLTHROUGH
            # If it failed the above, it's Risky (either unprofitable or unsafe balance sheet)
            else:
                return "Risky"

        # Check if we have cached data for this stock that's not expired
        cached_data = cache.get(ticker)
        if cached_data and (current_time - cached_data["timestamp"] < expiry_seconds):
            if cached_data.get("roic") == -999:
                continue  # Skip if previous fetch failed

        # FETCH NEW DATA using yfinance
        try:
            stock = yf.Ticker(ticker)
            fin = stock.financials  # Income statement data
            bs = stock.balance_sheet  # Balance sheet data

            # Check if Yahoo actually gave us data
            if fin.empty or bs.empty:
                print(f"   No data for {ticker} (skipping)")
                continue

            # --- CALCULATE 4-YEAR AVERAGE OPERATING MARGIN ---
            # We look at multiple years to see if profitability is consistent
            try:
                # Try to get Operating Income (might be called different things)
                if "Operating Income" in fin.index:
                    op_income_history = fin.loc["Operating Income"]
                elif "EBIT" in fin.index:
                    op_income_history = fin.loc["EBIT"]
                else:
                    op_income_history = pd.Series([0])

                # Get Revenue for same periods
                revenue_history = fin.loc["Total Revenue"]

                # Calculate margin for each year: Operating Income / Revenue
                yearly_margins = (op_income_history / revenue_history).dropna()

                if len(yearly_margins) > 0:
                    avg_margin = yearly_margins.mean()  # Average of all years

                    # Check if average meets our thresholds
                    is_fortress_margin = avg_margin > FORTRESS_MARGIN_THRESHOLD
                    is_positive_margin = avg_margin > 0
                else:
                    is_fortress_margin = False
                    is_positive_margin = False

            except Exception as e:
                is_fortress_margin = False
                is_positive_margin = False

            # --- STANDARD CALCULATIONS ---
            def get_item(df, keys):
                """Helper to get values that might have different names."""
                for k in keys:
                    if k in df.index:
                        return df.loc[k].iloc[0]
                return 0

            # Get needed values
            ebit = get_item(fin, ["EBIT", "Operating Income", "Pretax Income"])
            int_exp = get_item(
                fin, ["Interest Expense", "Interest Expense Non Operating"]
            )
            total_assets = get_item(bs, ["Total Assets"])
            curr_liab = get_item(
                bs, ["Current Liabilities", "Total Current Liabilities"]
            )

            # Calculate Interest Coverage Ratio
            # This tells us if the company can pay its interest bills
            int_exp = abs(int_exp)  # Make positive (expenses are often negative)
            if int_exp == 0:
                int_cov = 100  # No debt = infinite coverage, use 100 as placeholder
            else:
                int_cov = ebit / int_exp

            # Calculate ROIC (Return on Invested Capital)
            # Shows how efficiently management uses capital
            invested_cap = total_assets - curr_liab  # Simplified invested capital
            if invested_cap <= 0:
                roic = 0
            else:
                roic = ebit / invested_cap

            # Calculate Altman Z-Score
            base_row = survivor_df[survivor_df["Ticker"] == ticker].iloc[0]
            mkt_cap_raw = (
                base_row["Mkt Cap (B)"] * 1_000_000_000
            )  # Convert back to dollars
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)

            # Store metrics in cache for future use
            metrics = {
                "timestamp": current_time,
                "z_score": round(z, 2),
                "roic": roic,
                "int_cov": round(int_cov, 2),
            }
            cache[ticker] = metrics

            # Determine final tier based on all metrics
            tier = determine_tier_history(
                metrics, is_fortress_margin, is_positive_margin
            )

            # Add all data to our final results
            final_data.append(
                {
                    "Ticker": ticker,
                    "Tier": tier,
                    "Price": base_row["Price"],
                    "P/E": base_row["P/E"],
                    "Sector": base_row["Sector"],
                    "Z-Score": round(z, 2),
                    "ROIC %": round(roic * 100, 2),
                    "Op Margin %": base_row["Op Margin %"],
                    "Avg Margin (4Y)": (
                        round(avg_margin * 100, 2) if "avg_margin" in locals() else 0
                    ),
                    "Curr Ratio": base_row["Curr Ratio"],
                    "Int Cov": round(int_cov, 2),
                    "Mkt Cap (B)": base_row["Mkt Cap (B)"],
                }
            )

        except Exception as e:
            continue  # Skip if any error

    # Save updated cache to file
    save_cache(cache)

    return pd.DataFrame(final_data)

In [6]:
# =============================================================================
# CELL 6: MAIN EXECUTION
# =============================================================================
# This is where we actually RUN all the functions we defined above.
# The 'if __name__ == "__main__":' is a Python convention - it means
# "only run this code if this file is being run directly, not imported"

if __name__ == "__main__":
    # STEP 1: Get list of all US stocks
    tickers = get_combined_universe()

    if len(tickers) > 0:
        # STEP 2: Apply quick filters
        survivors_df = get_initial_survivors(tickers)

        if not survivors_df.empty:
            print(
                f"\n Step 2 Complete. {len(survivors_df)} stocks passed basic filters."
            )

            # STEP 3: Deep financial analysis
            final_results = get_advanced_metrics(survivors_df)

            if not final_results.empty:
                # Sort results: First by Tier (alphabetically), then by Z-Score (highest first)
                final_results = final_results.sort_values(
                    by=["Tier", "Z-Score"],
                    ascending=[True, False],  # True = A-Z, False = highest first
                )

                # Split into three categories based on tier
                fortress_df = final_results[final_results["Tier"] == "Fortress"].copy()
                strong_df = final_results[final_results["Tier"] == "Strong"].copy()
                risky_df = final_results[final_results["Tier"] == "Risky"].copy()

                # Save to CSV files (spreadsheet format)
                try:
                    fortress_df.to_csv(
                        FORTRESS_CSV, index=False
                    )  # index=False means don't save row numbers
                    strong_df.to_csv(STRONG_CSV, index=False)
                    risky_df.to_csv(RISKY_CSV, index=False)

                    print("\n" + "=" * 60)
                    print("RESULTS GENERATED")
                    print("=" * 60)
                    print(
                        f"1. FORTRESS ({len(fortress_df)}): Saved to '{FORTRESS_CSV}'"
                    )
                    print(f"2. STRONG   ({len(strong_df)}): Saved to '{STRONG_CSV}'")
                    print(f"3. RISKY    ({len(risky_df)}): Saved to '{RISKY_CSV}'")

                except Exception as e:
                    print(f"\n  Error Saving Files: {e}")

                # Set display options for pandas (so we can see more data)
                pd.set_option("display.max_rows", 500)
                pd.set_option("display.max_columns", 20)
                pd.set_option("display.width", 1000)

                # Show preview of fortress stocks
                print("\n--- FORTRESS PREVIEW ---")
                print(fortress_df.head(15))  # .head(15) shows first 15 rows
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Fetching Canadian Universe (TSX & TSX-V) ---
   -> Attempting to fetch official TMX list...
   -> Success: Found 1082 stocks via TMX.

--- STEP 2: Running 'Lightweight' Filter on 1082 stocks ---
 -> Processing Batch 1/3...

 Step 2 Complete. 108 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 108 Survivors ---
 -> Analyzing 1/108: ITR.V...
 -> Analyzing 21/108: ELD.TO...
 -> Analyzing 41/108: MTL.TO...
 -> Analyzing 61/108: FAR.TO...
   No data for RGSI.TO (skipping)
 -> Analyzing 81/108: EIF.TO...
 -> Analyzing 101/108: EFX.TO...

RESULTS GENERATED
1. FORTRESS (55): Saved to 'YfinanceDataDump\fortress_stocks_ca.csv'
2. STRONG   (21): Saved to 'YfinanceDataDump\strong_stocks_ca.csv'
3. RISKY    (31): Saved to 'YfinanceDataDump\risky_stocks_ca.csv'

--- FORTRESS PREVIEW ---
     Ticker      Tier    Price    P/E             Sector  Z-Score  ROIC %  Op Margin %  Avg Margin (4Y)  Curr Ratio  Int Cov  Mkt Cap (B)
41   WPM.TO  Fortress  187.850  61.59    Ba

In [7]:
# =============================================================================
# CELL 7: STEP 4 - ANALYST RATINGS FILTER
# =============================================================================
# This adds another layer of analysis: what do professional analysts think?
# Analyst Rating Scale (typically):
#   1.0 = Strong Buy
#   2.0 = Buy
#   3.0 = Hold
#   4.0 = Sell
#   5.0 = Strong Sell


def get_analyst_fortress_from_var(df_input):
    """
    Fetch analyst ratings for stocks and filter for those rated "Buy" or better.

    Args:
        df_input: DataFrame of stocks to analyze (usually fortress_df)

    Returns:
        DataFrame: Stocks with analyst ratings and upside potential
    """
    working_df = df_input.copy()
    tickers = working_df["Ticker"].tolist()

    print(f"\n--- STEP 4: Fetching Analyst Ratings for {len(tickers)} Stocks ---")
    print("    (Fetching serially to avoid throttling...)")

    analyst_data = []

    for i, ticker in enumerate(tickers):
        # Print progress every 10 stocks
        if i % 10 == 0:
            print(f" -> Analyst Scan {i+1}/{len(tickers)}: {ticker}...")

        try:
            stock = yf.Ticker(ticker)
            info = stock.info  # Get all available info for the stock

            # Extract analyst recommendation (1.0 to 5.0 scale)
            rec_mean = info.get("recommendationMean")
            target_price = info.get("targetMeanPrice")  # Average price target
            current_price = info.get("currentPrice")

            # Filter: Only keep stocks rated 2.5 or better (Buy or Strong Buy)
            if rec_mean is None or rec_mean > 2.5:
                continue

            # Calculate upside potential
            # Upside = (Target - Current) / Current * 100
            upside = 0
            if target_price and current_price:
                upside = round(
                    ((target_price - current_price) / current_price) * 100, 2
                )

            # Get the original data and add new columns
            base_row = working_df[working_df["Ticker"] == ticker].iloc[0].to_dict()
            base_row["Analyst_Rating"] = rec_mean
            base_row["Target_Price"] = target_price
            base_row["Upside_%"] = upside

            analyst_data.append(base_row)
            time.sleep(0.2)  # Small delay to not be timed out of Yahoo's servers

        except Exception:
            continue

    return pd.DataFrame(analyst_data)


# ==========================================
# EXECUTE THE ANALYST FILTER
# ==========================================

# Check if fortress_df exists from the previous step
if "fortress_df" in locals() and not fortress_df.empty:

    # Run the function
    Analyst_Fortress_DF = get_analyst_fortress_from_var(fortress_df)

    if not Analyst_Fortress_DF.empty:
        # Sort by highest upside potential
        Analyst_Fortress_DF = Analyst_Fortress_DF.sort_values(
            by="Upside_%", ascending=False
        )

        print("\n  Analyst Scan Complete!")
        print(f"Found {len(Analyst_Fortress_DF)} stocks with Buy Ratings (Score < 2.5)")

        # Save to CSV
        Analyst_Fortress_DF.to_csv(ANALYST_CSV, index=False)
        print(f"Saved to '{ANALYST_CSV}'")

        # Show top picks
        cols = ["Ticker", "Price", "Analyst_Rating", "Upside_%", "Target_Price", "Tier"]
        print(Analyst_Fortress_DF[cols].head(20))
    else:
        print("No stocks passed the Analyst filter.")
else:
    print("  'fortress_df' not found. Please run Steps 1-3 first.")


--- STEP 4: Fetching Analyst Ratings for 55 Stocks ---
    (Fetching serially to avoid throttling...)
 -> Analyst Scan 1/55: WPM.TO...
 -> Analyst Scan 11/55: DNG.TO...
 -> Analyst Scan 21/55: LIF.TO...
 -> Analyst Scan 31/55: RUS.TO...
 -> Analyst Scan 41/55: ENGH.TO...
 -> Analyst Scan 51/55: STGO.TO...

  Analyst Scan Complete!
Found 47 stocks with Buy Ratings (Score < 2.5)
Saved to 'YfinanceDataDump\Analyst_Fortress_Picks_ca.csv'
      Ticker    Price  Analyst_Rating  Upside_%  Target_Price      Tier
45    CMG.TO    5.195         2.42857     32.68      6.892860  Fortress
7    GRGD.TO   76.960         1.53846     31.65    101.307690  Fortress
29    VNP.TO   20.620         1.40000     25.13     25.788757  Fortress
9     DNG.TO    6.245         1.00000     24.24      7.758751  Fortress
36    STN.TO  137.520         1.72727     22.71    168.818180  Fortress
16    ATZ.TO  126.260         1.46154     19.91    151.416670  Fortress
12    TXG.TO   71.130         1.50000     19.06     84.62

In [8]:
# =============================================================================
# CELL 8: BUFFETT VALUE SCAN
# =============================================================================
# This filter looks for stocks Warren Buffett might like:
# - Trading BELOW book value (P/B < 1.0) means you're buying $1 of assets for less than $1
# - Positive Return on Equity (ROE) shows the company is profitable
# - Reasonable debt levels (Debt/Equity < 100%)


def get_buffett_value_picks(df_input):
    """
    Find deep value stocks trading below their book value.

    Book Value = Total Assets - Total Liabilities
    P/B Ratio = Stock Price / Book Value per Share

    If P/B < 1.0, you're theoretically buying the company for less than
    what it would be worth if you sold all its assets and paid all debts.

    Args:
        df_input: DataFrame of stocks to analyze

    Returns:
        DataFrame: Deep value stocks sorted by P/B ratio
    """
    print(f"\n--- STEP 5: Warren Buffett 'Below NAV' Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print(
        "    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%"
    )

    tickers = df_input["Ticker"].tolist()
    buffett_candidates = []

    # Process in chunks for speed
    chunk_size = 250
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # Get key statistics and financial data
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get("defaultKeyStatistics", {})
                        fin = data[symbol].get("financialData", {})

                        # 1. Price to Book < 1.0 (The Core "Value" Rule)
                        pb = stats.get("priceToBook")
                        if pb is None or pb >= 1.0 or pb <= 0:
                            continue

                        # 2. Positive ROE (Return on Equity - company makes money)
                        roe = fin.get("returnOnEquity", 0)
                        if roe is None or roe <= 0:
                            continue

                        # 3. Reasonable Debt (not overleveraged)
                        de = fin.get("debtToEquity", 0)
                        if de is None or de > 100:
                            continue

                        # Get base data and add value metrics
                        base_row = (
                            df_input[df_input["Ticker"] == symbol].iloc[0].to_dict()
                        )
                        base_row["P/B Ratio"] = round(pb, 2)
                        base_row["ROE %"] = round(roe * 100, 2)
                        base_row["Debt/Eq %"] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except:
                        continue
        except:
            continue

    return pd.DataFrame(buffett_candidates)


# Run the Buffett scan
if "final_results" in locals() and not final_results.empty:
    Buffett_Value_DF = get_buffett_value_picks(final_results)

    if not Buffett_Value_DF.empty:
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by="P/B Ratio", ascending=True)
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)

        print("\n" + "=" * 60)
        print("BUFFETT SCAN COMPLETE")
        print("=" * 60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks")

        cols = ["Ticker", "Price", "P/B Ratio", "ROE %", "Debt/Eq %", "Sector", "Tier"]
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head(20))
    else:
        print("\n  No stocks passed the Buffett filter.")
else:
    print(" 'final_results' not found. Please run Steps 1-3 first.")


--- STEP 5: Warren Buffett 'Below NAV' Scan ---
    Scanning 107 candidates for Deep Value...
    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%

BUFFETT SCAN COMPLETE
Found 1 Deep Value Stocks

--- DEEP VALUE PICKS ---
    Ticker  Price  P/B Ratio  ROE %  Debt/Eq %  Sector   Tier
0  MATR.TO   8.32       0.66   3.66      80.14  Energy  Risky


In [9]:
# =============================================================================
# CELL 9: INSIDER TRADING FILTER
# =============================================================================
# Insiders (executives, directors, major shareholders) sometimes have better
# information about their company. When they BUY their own stock, it can be
# a positive sign. When they SELL, it might be neutral (paying taxes) or negative.

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)


def filter_for_insider_buying(tickers):
    """
    Find stocks where company insiders are NET BUYERS.

    Net Buying = Total shares bought - Total shares sold
    If positive, insiders are accumulating shares (bullish signal).

    Args:
        tickers: List of stock symbols to check

    Returns:
        DataFrame: Stocks with positive insider buying
    """
    print(f" Scanning {len(tickers)} stocks for Insider Buying...")
    insider_picks = []

    # Process in smaller chunks to avoid timeout
    chunk_size = 25
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)

            # Fetch insider transaction data
            df_insiders = yq.insider_transactions

            # Fetch current prices
            price_data = yq.price

            # Skip if no data
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, "reset_index"):
                continue

            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders["symbol"].values:
                    continue

                # Get transactions for this stock
                stock_tx = df_insiders[df_insiders["symbol"] == symbol].copy()

                # Separate purchases and sales
                # The text typically says "Purchase" or "Sale"
                buys = stock_tx[
                    stock_tx["transactionText"]
                    .astype(str)
                    .str.contains("Purchase", case=False, na=False)
                ]
                sells = stock_tx[
                    stock_tx["transactionText"]
                    .astype(str)
                    .str.contains("Sale", case=False, na=False)
                ]

                # Calculate total buy/sell volume
                buy_vol = buys["shares"].sum() if not buys.empty else 0
                sell_vol = sells["shares"].sum() if not sells.empty else 0

                # Get current price
                current_price = None
                try:
                    if isinstance(price_data, dict) and symbol in price_data:
                        current_price = price_data[symbol].get(
                            "regularMarketPrice", None
                        )
                except:
                    current_price = None

                # Only keep if net buying is positive
                if buy_vol > sell_vol:
                    insider_picks.append(
                        {
                            "Ticker": symbol,
                            "Current_Price": current_price,
                            "Insider_Buys_Count": len(buys),
                            "Net_Shares_Bought": buy_vol - sell_vol,
                        }
                    )

        except Exception as e:
            continue

    return pd.DataFrame(insider_picks)


# Run the insider filter
if "fortress_df" in locals() and not fortress_df.empty:
    target_tickers = fortress_df["Ticker"].tolist()
else:
    print("'fortress_df' not found.")

Fortress_insiders = filter_for_insider_buying(target_tickers)
print(f"Created 'Fortress_insiders' with {len(Fortress_insiders)} rows.")
display(Fortress_insiders)

 Scanning 55 stocks for Insider Buying...
Created 'Fortress_insiders' with 33 rows.


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
0,DPM.TO,46.770,81,8453771.0
1,KNT.TO,26.070,1,8333.0
2,WDO.TO,25.650,8,706100.0
3,GRGD.TO,76.950,32,648000.0
4,DNG.TO,6.245,136,159600.0
5,ALS.TO,45.720,9,54100.0
6,TXG.TO,71.110,23,825769.0
7,ATZ.TO,126.280,62,471200.0
8,PAAS.TO,77.620,21,1650770.0
9,APM.TO,10.430,38,2081700.0


In [10]:
# ==========================================
# 10. ANALYST FILTER FUNCTION FOR INSIDER PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(df_insiders, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if df_insiders.empty:
        return df_insiders

    tickers = df_insiders["Ticker"].tolist()

    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data

        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and "recommendationMean" in data:
                    score = data.get("recommendationMean")

                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append(
                            {
                                "Ticker": t,
                                "Analyst_Score": score,
                                "Analyst_Verdict": data.get("recommendationKey", "N/A"),
                            }
                        )

        df_analyst = pd.DataFrame(analyst_data)

        if df_analyst.empty:
            print(" No Analyst ratings found for these tickers.")
            return df_insiders  # Return original if no data found

        # Merge with the Insider DataFrame
        merged = pd.merge(df_insiders, df_analyst, on="Ticker", how="inner")

        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged["Analyst_Score"] <= max_score].copy()

        print(
            f"  Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy)."
        )
        return final_df.sort_values(by="Analyst_Score", ascending=True)

    except Exception as e:
        print(f"  Error in Analyst Filter: {e}")
        return df_insiders


# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if "fortress_df" in locals() and not fortress_df.empty:
    target_tickers = fortress_df["Ticker"].tolist()
else:
    # Backup list just in case need to update periodically
    target_tickers = [
        "PET.TO",
        "MFI.TO",
        "TXG.TO",
        "SAP.TO",
        "PAAS.TO",
        "NEO.TO",
        "WPM.TO",
        "FNV.TO",
        "LUG.TO",
        "DPM.TO",
        "ASM.TO",
        "PNG.V",
        "DSG.TO",
        "KNT.TO",
        "GGD.TO",
        "GRGD.TO",
        "WDO.TO",
        "OGC.TO",
        "DNG.TO",
        "CLS.TO",
    ]

# B. Run Insider Filter
insider_winners = filter_for_insider_buying(target_tickers)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with Data Wrangler
if not insider_winners.empty:
    Fortress_insiders_Analyst_buy = filter_for_analyst_ratings(
        insider_winners, max_score=2.5
    )
else:
    Fortress_insiders_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_insiders_Analyst_buy.empty:
    print(
        f"\n  Final List: {len(Fortress_insiders_Analyst_buy)} stocks (Fortress + Insider Buying + Analyst Buy Rating)"
    )
    display(Fortress_insiders_Analyst_buy)
else:
    print("No stocks passed all filters.")

 Scanning 55 stocks for Insider Buying...
  Analyst Filter: 30 -> 29 stocks (Min Rating: Buy).

  Final List: 29 stocks (Fortress + Insider Buying + Analyst Buy Rating)


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought,Analyst_Score,Analyst_Verdict
4,DNG.TO,6.245,136,159600.0,1.00000,strong_buy
20,NWC.TO,49.420,40,238657.0,1.25000,strong_buy
9,APM.TO,10.430,38,2081700.0,1.33333,strong_buy
7,ATZ.TO,126.270,62,471200.0,1.46154,strong_buy
6,TXG.TO,71.110,23,825769.0,1.50000,strong_buy
1,KNT.TO,26.070,1,8333.0,1.50000,strong_buy
19,CEU.TO,13.580,24,33626038.0,1.50000,strong_buy
0,DPM.TO,46.770,81,8453771.0,1.50000,strong_buy
13,GIL.TO,88.630,105,1426104.0,1.53846,buy
3,GRGD.TO,76.950,32,648000.0,1.53846,buy


In [11]:
# =============================================================================
# CELL 11: BURRY EV/EBITDA FILTER For Fortress Stocks
# =============================================================================
# EV/EBITDA is a valuation metric popular with hedge fund manager Michael Burry.
#
# EV = Enterprise Value = Market Cap + Debt - Cash
#      (What it would cost to buy the whole company)
#
# EBITDA = Earnings Before Interest, Taxes, Depreciation, and Amortization
#          (Proxy for operating cash flow)
#
# EV/EBITDA tells you how many years of cash flow it would take to buy the company.
# Lower is better (cheaper stock).
# We compare each stock to its SECTOR AVERAGE to find relative value.


def filter_burry_ev_ebitda(df_input):
    """
    Find stocks trading at a discount to their sector average.

    Logic: A stock with EV/EBITDA of 8x is "cheap" in a sector
           where the average is 15x.

    Args:
        df_input: DataFrame of stocks to analyze (usually fortress_df)

    Returns:
        DataFrame: Stocks cheaper than their sector average
    """
    if df_input is None or df_input.empty:
        print(" Input DataFrame is empty.")
        return pd.DataFrame()

    print(f"Analyzing EV/EBITDA for {len(df_input)} Fortress stocks...")

    tickers = df_input["Ticker"].tolist()

    # Fetch data in bulk
    try:
        yq = Ticker(tickers, asynchronous=True)
        data = yq.get_modules("defaultKeyStatistics financialData summaryDetail")
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

    ev_data = []

    for ticker in tickers:
        try:
            ticker_data = data.get(ticker, {})
            if isinstance(ticker_data, str):
                continue

            stats = ticker_data.get("defaultKeyStatistics", {})
            fin_data = ticker_data.get("financialData", {})
            summary = ticker_data.get("summaryDetail", {})

            # Try to get pre-calculated EV/EBITDA
            ev_ebitda = stats.get("enterpriseToEbitda")

            # If not available, calculate manually
            if ev_ebitda is None:
                try:
                    market_cap = summary.get("marketCap")
                    total_debt = fin_data.get("totalDebt")
                    total_cash = fin_data.get("totalCash")
                    ebitda = fin_data.get("ebitda")

                    if all(
                        v is not None
                        for v in [market_cap, total_debt, total_cash, ebitda]
                    ):
                        if ebitda != 0:
                            # EV = Market Cap + Debt - Cash
                            enterprise_value = market_cap + total_debt - total_cash
                            ev_ebitda = enterprise_value / ebitda
                except:
                    pass

            # Only include positive values (profitable companies)
            if ev_ebitda is not None and ev_ebitda > 0:
                ev_data.append({"Ticker": ticker, "EV/EBITDA": round(ev_ebitda, 2)})
        except:
            continue

    df_vals = pd.DataFrame(ev_data)

    if df_vals.empty:
        print("Could not retrieve EV/EBITDA data.")
        return pd.DataFrame()

    # Merge with sector data
    merged_df = pd.merge(df_input, df_vals, on="Ticker", how="inner")

    # Calculate sector averages
    print("\n--- SECTOR AVERAGES (EV/EBITDA) ---")
    sector_stats = merged_df.groupby("Sector")["EV/EBITDA"].mean().reset_index()
    sector_stats.rename(columns={"EV/EBITDA": "Sector_Avg_EV_EBITDA"}, inplace=True)
    sector_stats["Sector_Avg_EV_EBITDA"] = sector_stats["Sector_Avg_EV_EBITDA"].round(2)

    print(sector_stats.to_string(index=False))

    # Merge with sector averages
    final_df = pd.merge(merged_df, sector_stats, on="Sector", how="left")

    # Filter: Keep only stocks CHEAPER than sector average
    burry_picks = final_df[
        final_df["EV/EBITDA"] < final_df["Sector_Avg_EV_EBITDA"]
    ].copy()

    # Calculate discount percentage
    # Discount = 1 - (Stock EV/EBITDA / Sector Average)
    burry_picks["Discount_%"] = round(
        (1 - (burry_picks["EV/EBITDA"] / burry_picks["Sector_Avg_EV_EBITDA"])) * 100, 2
    )

    # Sort by biggest discount
    burry_picks = burry_picks.sort_values(by="Discount_%", ascending=False)

    return burry_picks


# Run the Burry filter
if "fortress_df" in locals() and not fortress_df.empty:
    Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)

    if not Fortress_Burry_EV_EBITDA.empty:
        print(f"\n✅ Found {len(Fortress_Burry_EV_EBITDA)} Undervalued Stocks")
        display(
            Fortress_Burry_EV_EBITDA[
                [
                    "Ticker",
                    "Sector",
                    "Price",
                    "EV/EBITDA",
                    "Sector_Avg_EV_EBITDA",
                    "Discount_%",
                    "Tier",
                ]
            ]
        )

Analyzing EV/EBITDA for 55 Fortress stocks...

--- SECTOR AVERAGES (EV/EBITDA) ---
            Sector  Sector_Avg_EV_EBITDA
   Basic Materials                 20.31
 Consumer Cyclical                 18.99
Consumer Defensive                 12.84
            Energy                  6.46
       Industrials                 17.18
        Technology                 17.83

✅ Found 32 Undervalued Stocks


,Ticker,Sector,Price,EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Tier
34,FVI.TO,Basic Materials,14.350,6.75,20.31,66.77,Fortress
14,MSA.TO,Basic Materials,6.540,6.97,20.31,65.68,Fortress
7,WDO.TO,Basic Materials,25.660,7.14,20.31,64.84,Fortress
47,STGO.TO,Basic Materials,2.005,7.82,20.31,61.50,Fortress
51,FAR.TO,Basic Materials,2.540,8.15,20.31,59.87,Fortress
38,ENGH.TO,Technology,19.555,7.63,17.83,57.21,Fortress
25,CG.TO,Basic Materials,22.380,9.12,20.31,55.10,Fortress
10,DNG.TO,Basic Materials,6.245,9.19,20.31,54.75,Fortress
13,TXG.TO,Basic Materials,71.130,11.39,20.31,43.92,Fortress
29,RUS.TO,Industrials,48.300,9.64,17.18,43.89,Fortress


In [12]:
# ==========================================
# 12. ANALYST FILTER FUNCTION FOR Burry EV/EBITDA PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(Fortress_Burry_EV_EBITDA, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if Fortress_Burry_EV_EBITDA.empty:
        return Fortress_Burry_EV_EBITDA

    tickers = Fortress_Burry_EV_EBITDA["Ticker"].tolist()

    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data

        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and "recommendationMean" in data:
                    score = data.get("recommendationMean")

                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append(
                            {
                                "Ticker": t,
                                "Analyst_Score": score,
                                "Analyst_Verdict": data.get("recommendationKey", "N/A"),
                            }
                        )

        df_analyst = pd.DataFrame(analyst_data)

        if df_analyst.empty:
            print("⚠️ No Analyst ratings found for these tickers.")
            return Fortress_Burry_EV_EBITDA  # Return original if no data found

        # Merge with the Fortress DataFrame
        merged = pd.merge(
            Fortress_Burry_EV_EBITDA, df_analyst, on="Ticker", how="inner"
        )

        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged["Analyst_Score"] <= max_score].copy()

        print(
            f"✅ Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy)."
        )
        return final_df.sort_values(by="Analyst_Score", ascending=True)

    except Exception as e:
        print(f"❌ Error in Analyst Filter: {e}")
        return Fortress_Burry_EV_EBITDA


# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if "Fortress_Burry_EV_EBITDA" in locals() and not Fortress_Burry_EV_EBITDA.empty:
    target_tickers = Fortress_Burry_EV_EBITDA["Ticker"].tolist()


# B. Run Insider Filter
Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with Data Wrangler
if not Fortress_Burry_EV_EBITDA.empty:
    Fortress_Burry_Analyst_buy = filter_for_analyst_ratings(
        Fortress_Burry_EV_EBITDA, max_score=2.5
    )
else:
    Fortress_Burry_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_Burry_Analyst_buy.empty:
    print(
        f"\n🚀 Final List: {len(Fortress_Burry_Analyst_buy)} stocks (Fortress + Burry + Analyst Buy Rating)"
    )
    display(Fortress_Burry_Analyst_buy)
else:
    print("No stocks passed all filters.")

Analyzing EV/EBITDA for 55 Fortress stocks...

--- SECTOR AVERAGES (EV/EBITDA) ---
            Sector  Sector_Avg_EV_EBITDA
   Basic Materials                 20.31
 Consumer Cyclical                 18.99
Consumer Defensive                 12.84
            Energy                  6.46
       Industrials                 17.18
        Technology                 17.83
✅ Analyst Filter: 27 -> 27 stocks (Min Rating: Buy).

🚀 Final List: 27 stocks (Fortress + Burry + Analyst Buy Rating)


,Ticker,Tier,Price,P/E,Sector,Z-Score,ROIC %,Op Margin %,Avg Margin (4Y),Curr Ratio,Int Cov,Mkt Cap (B),EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Analyst_Score,Analyst_Verdict
1,MSA.TO,Fortress,6.540,8.84,Basic Materials,8.86,31.48,37.50,22.59,1.691,18.14,1.93,6.97,20.31,65.68,1.00000,strong_buy
4,DNG.TO,Fortress,6.245,12.25,Basic Materials,10.35,25.90,7.90,9.47,4.633,68.99,0.26,9.19,20.31,54.75,1.00000,strong_buy
14,NWC.TO,Fortress,49.450,16.82,Consumer Defensive,4.77,16.76,9.46,8.38,2.258,11.20,2.36,9.00,12.84,29.91,1.25000,strong_buy
12,APM.TO,Fortress,10.430,14.69,Basic Materials,7.80,15.31,33.71,5.28,2.678,6.17,1.55,13.74,20.31,32.35,1.33333,strong_buy
23,SVM.TO,Fortress,14.220,94.80,Basic Materials,7.43,10.35,42.70,30.35,4.588,22.11,3.13,19.63,20.31,3.35,1.33333,strong_buy
19,ARIS.TO,Fortress,25.330,68.46,Basic Materials,3.81,5.73,40.92,29.55,2.415,3.90,5.14,16.82,20.31,17.18,1.40000,strong_buy
5,TXG.TO,Fortress,71.130,15.07,Basic Materials,9.55,19.55,43.61,32.65,1.377,100.00,6.84,11.39,20.31,43.92,1.50000,strong_buy
18,KNT.TO,Fortress,26.070,18.23,Basic Materials,27.06,30.81,68.87,35.98,3.285,456.18,6.34,16.58,20.31,18.37,1.50000,strong_buy
22,GIL.TO,Fortress,88.600,20.51,Consumer Cyclical,6.52,20.83,23.23,19.48,2.008,7.02,16.41,17.13,18.99,9.79,1.53846,buy
7,OGC.TO,Fortress,45.280,19.03,Basic Materials,12.32,12.37,36.34,20.19,1.274,12.14,10.34,11.57,20.31,43.03,1.54545,buy


In [13]:
# ==========================================
# 13. THE "DEEP VALUE" INTERSECTION (Buffett + Burry)
# ==========================================

# 1. Ensure we have the Buffett Data
if "Buffett_Value_DF" not in locals():
    print("Buffett Data not found. Running scan now...")
    if "get_buffett_value_picks" in globals() and "final_results" in locals():
        Buffett_Value_DF = get_buffett_value_picks(final_results)
    else:
        print("Missing 'final_results' or 'get_buffett_value_picks' function.")
        Buffett_Value_DF = pd.DataFrame()

# 2. Ensure we have the Burry Data
if "Fortress_Burry_EV_EBITDA" not in locals():
    print("Please run the Burry EV/EBITDA filter cell first.")
    Fortress_Burry_EV_EBITDA = pd.DataFrame()

# 3. THE MERGE (Finding the Overlap)
if not Buffett_Value_DF.empty and not Fortress_Burry_EV_EBITDA.empty:

    # Merge on Ticker to find stocks that appear in BOTH lists
    # We use an 'inner' join, which means "keep only if in both"
    Deep_Value_Intersection = pd.merge(
        Buffett_Value_DF[["Ticker", "P/B Ratio", "ROE %", "Debt/Eq %"]],
        Fortress_Burry_EV_EBITDA[
            [
                "Ticker",
                "Price",
                "Sector",
                "EV/EBITDA",
                "Sector_Avg_EV_EBITDA",
                "Discount_%",
                "Tier",
            ]
        ],
        on="Ticker",
        how="inner",
    )

    if not Deep_Value_Intersection.empty:
        print("\n" + "=" * 60)
        print(f"DEEP VALUE GEMS FOUND: {len(Deep_Value_Intersection)}")
        print("=" * 60)
        print(
            "Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)"
        )

        # Sort by the "Discount" (how cheap they are vs sector)
        Deep_Value_Intersection = Deep_Value_Intersection.sort_values(
            by="Discount_%", ascending=False
        )

        cols = [
            "Ticker",
            "Price",
            "Tier",
            "P/B Ratio",
            "EV/EBITDA",
            "Sector_Avg_EV_EBITDA",
            "Discount_%",
            "Sector",
        ]
        display(Deep_Value_Intersection[cols])

    else:
        print("\n No stocks passed BOTH filters.")
        print(
            "This means no stock is both 'Below Book Value' AND 'Cheaper than Sector Average' at the same time."
        )
        print(
            f"Buffett Count: {len(Buffett_Value_DF)} | Burry Count: {len(Fortress_Burry_EV_EBITDA)}"
        )

else:
    print("Cannot combine. One of the filters returned 0 results.")


 No stocks passed BOTH filters.
This means no stock is both 'Below Book Value' AND 'Cheaper than Sector Average' at the same time.
Buffett Count: 1 | Burry Count: 32


In [14]:
# =============================================================================
# CELL 14: THE "HIGH CONVICTION" INTERSECTION (BURRY + INSIDERS)
# =============================================================================
# This finds stocks that are BOTH:
# 1. Undervalued vs their Sector (Burry EV/EBITDA)
# 2. Being accumulated by Insiders (Insider Confidence)
#
# This is a powerful "Anti-Value Trap" filter. Insiders rarely buy
# sinking ships, even if they look cheap on paper.

# --- 1. DETECT BEST AVAILABLE DATAFRAMES ---

# Try to find the Insider list (preferring the one with Analyst ratings if available)
if (
    "Fortress_insiders_Analyst_buy" in locals()
    and not Fortress_insiders_Analyst_buy.empty
):
    df_insider_source = Fortress_insiders_Analyst_buy
    print("Using Analyst-Vetted Insider List.")
elif "Fortress_insiders" in locals() and not Fortress_insiders.empty:
    df_insider_source = Fortress_insiders
    print("Using Raw Insider List (No Analyst Check).")
else:
    df_insider_source = pd.DataFrame()
    print("Warning: No Insider data found.")

# Try to find the Burry list (preferring the one with Analyst ratings if available)
if "Fortress_Burry_Analyst_buy" in locals() and not Fortress_Burry_Analyst_buy.empty:
    df_burry_source = Fortress_Burry_Analyst_buy
    print("Using Analyst-Vetted Burry List.")
elif "Fortress_Burry_EV_EBITDA" in locals() and not Fortress_Burry_EV_EBITDA.empty:
    df_burry_source = Fortress_Burry_EV_EBITDA
    print("Using Raw Burry List (No Analyst Check).")
else:
    df_burry_source = pd.DataFrame()
    print("Warning: No Burry EV/EBITDA data found.")

# --- 2. PERFORM THE INTERSECTION ---

if not df_insider_source.empty and not df_burry_source.empty:

    # We merge on 'Ticker'. 'inner' means keep only stocks found in BOTH lists.
    # We use suffixes=('', '_drop') to handle duplicate columns (like Price)
    high_conviction_df = pd.merge(
        df_burry_source[
            [
                "Ticker",
                "Price",
                "Sector",
                "Tier",
                "EV/EBITDA",
                "Sector_Avg_EV_EBITDA",
                "Discount_%",
            ]
        ],
        df_insider_source[["Ticker", "Insider_Buys_Count", "Net_Shares_Bought"]],
        on="Ticker",
        how="inner",
    )

    if not high_conviction_df.empty:
        # Sort by Discount % (Deepest value first)
        high_conviction_df = high_conviction_df.sort_values(
            by="Discount_%", ascending=False
        )

        print("\n" + "=" * 60)
        print(f"🚀 HIGH CONVICTION WINNERS FOUND: {len(high_conviction_df)}")
        print("=" * 60)
        print("Criteria: Cheap vs Sector (Burry) + Insider Buying")

        # Reorder columns for readability
        cols = [
            "Ticker",
            "Price",
            "Sector",
            "Tier",
            "Discount_%",
            "EV/EBITDA",
            "Insider_Buys_Count",
            "Net_Shares_Bought",
        ]

        # Display nicely
        display(high_conviction_df[cols])

        # Optional: Save to CSV
        save_path = os.path.join(DATA_FOLDER, "High_Conviction_Picks_ca.csv")
        high_conviction_df.to_csv(save_path, index=False)
        print(f"\nSaved to: {save_path}")

    else:
        print(
            "\nNo overlap found. No stock is both 'Cheap vs Sector' AND 'Insider Buy' target."
        )
        print(
            f"Burry Count: {len(df_burry_source)} | Insider Count: {len(df_insider_source)}"
        )
else:
    print("\nCannot run intersection. Missing one or both source lists.")

Using Analyst-Vetted Insider List.
Using Analyst-Vetted Burry List.

🚀 HIGH CONVICTION WINNERS FOUND: 18
Criteria: Cheap vs Sector (Burry) + Insider Buying


,Ticker,Price,Sector,Tier,Discount_%,EV/EBITDA,Insider_Buys_Count,Net_Shares_Bought
16,FVI.TO,14.350,Basic Materials,Fortress,66.77,6.75,10,2116207.0
13,WDO.TO,25.660,Basic Materials,Fortress,64.84,7.14,8,706100.0
17,CG.TO,22.380,Basic Materials,Fortress,55.10,9.12,88,4428018.0
0,DNG.TO,6.245,Basic Materials,Fortress,54.75,9.19,136,159600.0
3,TXG.TO,71.130,Basic Materials,Fortress,43.92,11.39,23,825769.0
11,RUS.TO,48.300,Industrials,Fortress,43.89,9.64,139,2297083.0
7,CCL-B.TO,85.810,Consumer Cyclical,Fortress,42.92,10.84,4,627580.0
14,ELD.TO,56.615,Basic Materials,Fortress,37.91,12.61,119,5087258.0
6,FTT.TO,83.180,Industrials,Fortress,36.09,10.98,11,18282.0
15,CMG.TO,5.195,Technology,Fortress,34.10,11.75,28,1224000.0



Saved to: YfinanceDataDump\High_Conviction_Picks_ca.csv


In [15]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================


import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = [
    "CVE.TO",
    "BANC",
    "ONB",
    "UBER",
    "ADMA",
    "MIR",
    "APG",
    "SEI",
    "FLEX",
    "DD",
    "SVM",
]


def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")

    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []

    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()

            finviz_data.append(
                {
                    "Ticker": ticker,
                    "Recom": info.get("Recom", np.nan),
                    "Target_Price": info.get("Target Price", np.nan),
                }
            )
            time.sleep(0.5)

        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append(
                {"Ticker": ticker, "Recom": np.nan, "Target_Price": np.nan}
            )

    df_finviz = pd.DataFrame(finviz_data)

    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")

    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(
            ticker_list,
            period="1y",
            interval="1d",
            group_by="ticker",
            progress=False,
            threads=True,
        )
        yf_stats = []

        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=["Close"])
                if len(df) < 20:
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df["Close"].iloc[-1]
                prev_close = df["Close"].iloc[-2]

                high_52 = df["High"].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100

                change_pct = ((current_price - prev_close) / prev_close) * 100

                # Volatility (30-day Std Dev)
                volatility = df["Close"].pct_change().std() * 100

                # Relative Volume
                curr_vol = df["Volume"].iloc[-1]
                avg_vol = df["Volume"].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df["Close"].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100

                yf_stats.append(
                    {
                        "Ticker": ticker,
                        "Price": round(current_price, 2),
                        "Change_%": round(change_pct, 2),
                        "52W_MA": round(ma_52w, 2),  # <--- Added Here
                        "Drop_from_High_%": round(drop_from_high, 2),
                        "Volatility_%": round(volatility, 2),
                        "Rel_Volume": round(rel_vol, 2),
                    }
                )

            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue

        df_yf = pd.DataFrame(yf_stats)

    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on="Ticker", how="outer")
        else:
            master_df = df_finviz

        # Added '52W_MA' to this list so it displays in the final table
        cols = [
            "Ticker",
            "Price",
            "Change_%",
            "52W_MA",
            "Drop_from_High_%",
            "Recom",
            "Target_Price",
            "Rel_Volume",
            "Volatility_%",
        ]

        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()


# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if "Drop_from_High_%" in watchlist_df.columns:
        watchlist_df["Drop_from_High_%"] = pd.to_numeric(
            watchlist_df["Drop_from_High_%"], errors="coerce"
        )
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by="Drop_from_High_%", ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 11 stocks ---
1. Fetching Analyst Ratings from Finviz...
   Skipping Finviz for CVE.TO: HTTP error for URL https://finviz.com/quote.ashx?t=CVE.TO: 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=CVE.TO
2. Fetching Price & Volatility from yfinance...

--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
0,ADMA,17.74,-2.95,17.92,-30.89,1.00,30.00,0.83,3.17
10,UBER,84.17,-0.57,85.32,-17.47,1.47,111.17,0.74,2.40
6,MIR,27.17,3.50,20.29,-10.26,1.12,30.50,0.61,3.50
5,FLEX,66.60,4.34,49.31,-7.78,1.42,75.00,0.56,2.92
3,CVE.TO,25.22,-1.37,20.82,-3.55,NaN,NaN,0.62,2.54
8,SEI,55.45,6.59,33.57,-3.25,1.17,66.27,1.04,5.89
7,ONB,23.29,1.92,21.50,-2.46,1.85,26.23,0.65,2.10
4,DD,43.42,1.25,32.46,-1.69,1.48,47.88,0.69,2.23
1,APG,42.66,1.50,32.14,-1.39,1.45,44.40,0.56,1.93
9,SVM,10.23,1.59,5.15,-0.73,1.17,11.00,0.65,3.61


In [16]:
tickers_gemini = ["RUS.TO"]
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

# ==========================================
# SECURE CONFIGURATION
# ==========================================

# 1. Define the path to your key file
# If the file is in the same folder as this notebook, just use the filename.
KEY_FILE_PATH = (
    r"C:\Users\jdcc3\OneDrive - McMaster University\Gemini API Key\gemini_key.txt"
)


def load_api_key(filepath):
    """
    Reads the API key from a local file to avoid hardcoding it.
    """
    try:
        with open(filepath, "r") as f:
            # .strip() removes any accidental newlines or spaces
            return f.read().strip()
    except FileNotFoundError:
        print(f"Error: Could not find the file '{filepath}'")
        print("Please create a text file with your API key in it.")
        return None
    except Exception as e:
        print(f"Error reading key file: {e}")
        return None


# 2. Load the key and set the environment variable
api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("API Key loaded securely.")
else:
    print("CRITICAL: API Key not loaded. The script will fail.")


# ==========================================
# SENTIMENT ANALYSIS FUNCTION USING GROUNDING SEARCH GEMINI 3
# ==========================================
def analyze_sentiment_gemini_3(tickers_gemini, company_name=None):

    if not os.environ.get("GEMINI_API_KEY"):
        print("Stop: No API Key found.")
        return

    print(
        f"\nGemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}..."
    )

    # Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, thinking_level="HIGH"
        ),
        tools=[types.Tool(google_search=types.GoogleSearch())],
        response_modalities=["TEXT"],
    )

    prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini} ({company_name if company_name else 'the company'}).
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology. Look for contradictions between price action and news.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric? (Look for scandals, lawsuits, or product breakthroughs).
    2. **Analyst Shifts**: Are price targets moving UP or DOWN in the last week?
    3. **Institutional Flows**: Any reports of hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums vs. official guidance?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## Gemini 3 Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis (Why it goes up)
    * ...
    
    ### 2. The Bear Thesis (Why it goes down)
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance over last 3 years + expected performance)
    
    ### 4. Conclusion
    [Summary of whether the current price is a trap or an opportunity]
    """

    try:
        response = client.models.generate_content(
            model="gemini-3-flash-preview",  # Or 'gemini-3-flash-preview'
            contents=prompt,
            config=config,
        )
        display(Markdown(response.text))

    except Exception as e:
        print(f"Error: {e}")


# ==========================================
# EXECUTION
# ==========================================
# Only run this if the key loaded successfully
if os.environ.get("GEMINI_API_KEY"):
    analyze_sentiment_gemini_3(tickers_gemini, tickers_gemini)

API Key loaded securely.

Gemini 3 is thinking (High Reasoning Mode)... analyzing $['RUS.TO']...


## Gemini 3 Sentiment Report: ['RUS.TO']
**Reasoning Depth:** High
**Sentiment Score:** 8.2/10
**Verdict:** Buy / Strong Hold

### 1. The Bull Thesis (Why it goes up)
*   **M&A Catalyst Integration**: On January 2, 2026, Russel Metals successfully closed the acquisition of seven service centers from Kloeckner Metals for US$118.6 million. This strategic move is expected to add ~US$500 million in annual revenue and shift the company's revenue mix to over 50% from the U.S. market, reducing geographic concentration risk.
*   **Analyst Target Rerating**: Major financial institutions have aggressively raised price targets in the last 7 days. RBC Capital boosted its target to **C$51.00** (Jan 14, 2026), following National Bankshares’ target of **C$56.00**. The consensus suggests a ~10% upside from current 52-week highs.
*   **Infrastructure & Data Center Tailwinds**: The company is positioned as a primary beneficiary of the "North American Nation Building" cycle. Increased demand for structural steel in data centers, energy transition projects, and government-backed infrastructure is providing a floor for industrial volumes.
*   **Robust Dividend & Buyback Profile**: With a current yield of **3.6-3.8%** and a consistent quarterly payout of C$0.43, the company remains a "Dividend Stalwart." The active share buyback program ($14M repurchased in the most recent quarter) signals management’s belief that shares remain undervalued.

### 2. The Bear Thesis (Why it goes down)
*   **The "Lumpy Business" Risk**: Management warned of "lumpy" revenue and earnings patterns during the Q3 call. A miss on EPS (C$0.63 vs C$0.70 estimate) in late 2025 initially spooked the market, highlighting that financial results can be inconsistent quarter-to-quarter due to project timing.
*   **Seasonal Volatility**: Q4 results (expected Feb 2026) typically face seasonal headwinds in Western Canada and the U.S. due to fewer shipping days and construction pauses.
*   **Input Price Sensitivity**: While tariffs initially supported steel prices in early 2025, any significant drop in global steel or aluminum prices could squeeze margins in the Steel Distribution segment.

### 3. Deep Dive Analysis
*   **News Analysis**: Market sentiment is currently **euphoric** regarding the Kloeckner deal closure. Headlines in January 2026 have transitioned from "Mixed Earnings" in November to "Fresh Annual Peaks" in mid-January. There are no active scandals or lawsuits; the news cycle is dominated by expansion and credit upgrades (S&P upgraded RUS to investment-grade **BBB-**).
*   **Smart Money**: Institutional flows are constructive. The "Smart Money" has ignored the Q3 EPS miss, focusing instead on the **16% Return on Invested Capital (ROIC)**, which outperforms its U.S. peers. There has been no reported insider "dumping" despite the stock hitting 52-week highs, suggesting insiders expect the Kloeckner integration to drive further gains.
*   **Financial Statement Analysis**:
    *   **Historic (2023-2024)**: Revenue saw a moderate contraction (-5.4% in 2024) as post-pandemic pricing normalized.
    *   **2025 Performance**: A recovery year. YTD revenues increased 10% and EBITDA rose 13% compared to 2024. Gross margins improved by 100 basis points.
    *   **2026 Outlook**: Expected to be a breakout year for revenue due to the Samuel and Kloeckner acquisitions, with total capital deployment reaching ~C$1.9 billion.

### 4. Conclusion
The current price of **C$47.73** is **not a trap**, but a breakout validated by fundamentals. While the stock is trading at a premium compared to its 2024 levels, the "lumpy" EPS miss of Q3 2025 provided a consolidation period that has now resolved into a bullish trend. 

**Traders' "Whisper":** The whisper on forums is that the Kloeckner acquisition was "bought right" at a time when U.S. manufacturing is poised for a multi-year recovery. While official guidance remains conservative regarding Q4 seasonality, traders are positioning for a strong 2026 guidance update in February. 

**Final Verdict:** Accumulate on minor pullbacks. The combination of an investment-grade credit rating, strategic U.S. expansion, and a reliable 3.8% yield makes RUS.TO a high-quality industrial play.